## ChatCompletion API实践之多轮对话机器人

本章主要利用ChatCompletion API的使用技巧，以及message参数中system可以实现知识背景的设置；
让大模型浏览本地知识库内容，将其设定为System role的知识背景，实现一个多轮对话机器人的案例

1.初步测试
咨询大模型一个未经训练不知道的内容

In [17]:
from openai import OpenAI
client = OpenAI()
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "null"},
    {"role": "user", "content": '你知道《地球特派员》的电影里角色有哪些吗？'}
  ]
)

In [18]:
response.choices[0].message.content

'《地球特派员》是一部动画电影，也称为《Men in Black》。以下是电影中的一些角色：\n\n1. Agent K (凯)\n2. Agent J (杰)\n3. Frank the Pug (弗兰克)\n4. Agent L (艾丽)\n5. Zed (齐德)\n6. Edgar the Bug (埃德加)\n7. Laurel Weaver (劳瑞）\n8. Serleena (赛琳娜)\n\n这些是电影中比较重要的主要角色，还有其他一些次要角色和外星人角色。'

2.读取本地知识库
读取本地服务器提前准备好的知识库内容

In [19]:
with open('地球特派员.txt', 'r', encoding='utf-8') as f:
    chatCompletion_kg = f.read()

In [20]:
chatCompletion_kg

'电影：地球特派员 Elio\n导演: 阿德里安·莫利纳\n主演: 约纳斯·基布雷亚布 / 亚美莉卡·费雷拉\n类型: 喜剧 / 科幻 / 动画 / 冒险\n上映日期: 2024-03-01(美国)\n这是皮克斯宣布了第 28 部故事片，将聚焦于一个 11 岁的男孩（当然叫 Elio），他无意中被传送到太空，被误认为是另一个星球上的地球大使。'

3.问答测试
以本地知识库作为背景知识，让大模型回答相关问题

In [21]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": chatCompletion_kg},
    {"role": "user", "content": '你知道《地球特派员》的电影里角色有哪些吗？'}
  ]
)

In [22]:
response.choices[0].message.content

'当然！《地球特派员》电影中的主要角色包括：\n\n1. Elio：主角，一个 11 岁的男孩，被误认为是地球大使而被传送到另一个星球上。\n2. 太空探险家 Zara：另一个星球的居民，是 Elio 在外太空中遇到的重要角色。\n3. 机器人小助手 R1C4：协助 Elio 在另一个星球上生存和应对挑战的机器人伙伴。\n4. 地球特派员团队：一群来自不同星球的使者，与 Elio 一起合作展开冒险活动。\n\n这些角色将在影片中展开一系列有趣、惊险和感人的故事情节。'

4.实现多轮对话

利用本地知识库+循环处理实现简单的多轮对话效果

In [23]:
def chat_with_model(messages):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages
    )
    return response.choices[0].message.content

# 测试函数
def gpt_chat_with_model():
    # 初始问候
    messages = [
        {"role": "system", "content": chatCompletion_kg}, ##添加本地知识库
        {"role": "user", "content": "你好！"},
        {"role": "assistant", "content": "你好！我是一个AIGC智能助理，有什么问题我可以帮助你？"}
    ]
    print(chat_with_model(messages))

    # 进行对话
    while True:
        user_input = input("用户：")
        messages.append({"role": "user", "content": user_input})
        assistant_response = chat_with_model(messages)
        messages.append({"role": "assistant", "content": assistant_response})
        print("助理：" + assistant_response)

        # 判断是否结束对话
        if user_input.lower() == 'quit':
            break

In [25]:
gpt_chat_with_model()

电影《地球特派员 Elio》讲述了一个11岁男孩被误认为是地球大使的故事。你喜欢看这类喜剧、科幻和冒险题材的电影吗？
助理：这部电影的导演是阿德里安·莫利纳（Adrian Molina）。
助理：《地球特派员》的主演包括约纳斯·基布雷亚布（Jonas Kühnemann）和亚美莉卡·费雷拉（Amarilys Ferreira）。
助理：《地球特派员》预计于2024年3月1日在美国上映。
助理：《地球特派员》是一部喜剧、科幻、动画和冒险题材相结合的电影。
助理：如有任何其他问题，欢迎随时向我提问！祝您有美好的一天！如果需要帮助，请随时回来。再见！
